In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

In [3]:
sys.path.append(os.path.abspath(os.path.join("../../..")))

In [4]:
import pandas as pd

from notebooks.utils.constants import question_ids_map
from notebooks.utils.simplified_experiment import SimplifiedExperiment

/Users/david/eafit/eafit-tesis/.venv/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Retrieve answers file


In [5]:
answers_df = pd.read_excel("../../../data/asc_842/lease_agreements_info/answers.xlsx")

Get questions to send to the AI


In [6]:
question_id = question_ids_map["effective_date"]

In [7]:
template_string = """On what date was the contract executed? This could the date the document was signed, or if that date is unavailable its the date on which the agreement was entered into or is dated. Provide the answer in the answer_date field in the format YYYY-MM-DD.
Context: {context}
"""

## Variation 4

- gpt-4o-mini
- text-embedding-3-large
- structured outputs


In [8]:
experiment = SimplifiedExperiment(
    answers_df=answers_df,
    question_id=question_id,
    index_name="eafit-text-embedding-3-large",
    date_question=True,
    csv_results_filename="results/effective_date_prompt_tuning_v7.csv",
    template_string=template_string,
    retriever_question="""
    Question: What is the effective date of the contract?

    Instructions:
    1. Identify the signature date: Begin by checking the contract for a specific signature date. This is often listed near the end of the document, alongside the parties' signatures. If this date is present, it should generally be considered the effective date.
    2. Check for an explicitly stated effective date: If the contract does not have a signature date or it is unclear, look for any explicitly stated "effective date" or "date of execution" within the contract text, which may sometimes be noted at the beginning or in an introductory section.
    3. Determine fallback date: If neither a signature date nor an explicit effective date is provided, identify any other date that indicates when the agreement was entered into, such as a date mentioned in the title or heading.
    """,
    use_llm_filter=True,
    chunks_filename="chunks/effective_date_chunks_v7.txt",
)
experiment.run()

Error processing lease001: Failed to connect; did you specify the correct index name?
Lease: lease002
Answer: 2023-11-22
Real answer: 2024-03-01
INCORRECT
----------------------------------------------------
Lease: lease003
Answer: 2023-10-01
Real answer: 2021-11-01
INCORRECT
----------------------------------------------------
Lease: lease004
Answer: 2023-10-01
Real answer: 2015-06-20
INCORRECT
----------------------------------------------------
Lease: lease005
Answer: 1999-11-16
Real answer: 1995-03-31
INCORRECT
----------------------------------------------------
Lease: lease006
Answer: 2023-10-01
Real answer: 2021-08-01
INCORRECT
----------------------------------------------------
Lease: lease007
Answer: 2009-02-10
Real answer: 2009-02-10
CORRECT
----------------------------------------------------
Lease: lease008
Answer: 2023-10-01
Real answer: 2003-03-21
INCORRECT
----------------------------------------------------
Lease: lease010
Answer: 2018-06-29
Real answer: 2018-07-05
INC